In [12]:
#Monment , Hu Invarient Moment - 이것을 이용해서 매칭이 가능하다.

import cv2
import numpy as np
import imutils

src = cv2.imread("./data/momentTest.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret , blmage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
#맨 끝에 있는 외각선을 contours라고 한다.
#blmage = 바이너리 이미지
contours , _ = cv2.findContours(blmage, mode, method)

dst = src.copy()
#contours의 0 번째 요소를 넣는다.
cnt = contours[0]
#외각선을 그려준다.
cv2.drawContours(dst, [cnt], 0, (255,0,0), 3)

cv2.imshow("contour", dst)
cv2.waitKey()
cv2.destroyAllWindows()

#모멘트에서 나온 결과
M = cv2.moments(cnt)
#7개의 결과가 나오는데 invariant 인 베리언트 하다.
hu = cv2.HuMoments(M)

print("hu.shape= ", hu.shape)
print("hu=", hu)

#휴모먼트가 invarient하다 따라서
#앞에서 구한 모멘트를 45도로 회전하고 크기를 0.2로 하고 비교하는 것
angle = 45.0
scale = 0.2
cx = M["m10"]/M["m00"] #강의자료에 설명이 나와있음
cy = M["m01"]/M["m00"]
center = (cx , cy) #중점
t=(20,30)

A = cv2.getRotationMatrix2D(center, angle, scale)
A[:,2] += t #translation
cnt2 = cv2.transform(cnt, A)#적용하기 위해서는 transform을 적용한다.
cv2.drawContours(dst, [cnt2], 0, (0,255,0), 3 )

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

M2 = cv2.moments(cnt2)
hu2 = cv2.HuMoments(M2)
print("hu2.shape=", hu.shape)
print("hu2=", hu)


diffSum = np.sum(cv2.absdiff(hu,hu2))
print("diffSum1=", diffSum)

#이렇게 diffSum 을 통해서 매칭의 값을 구할 수 있다.

hu.shape=  (7, 1)
hu= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
hu2.shape= (7, 1)
hu2= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
diffSum1= 0.0003215707378081359


In [2]:
import cv2
import numpy as np
import imutils

#Hu로 매칭 시켜보기

ref_src = cv2.imread("./data/refShapes.jpg")
ref_gray = cv2.cvtColor(ref_src , cv2.COLOR_BGR2GRAY)
ret , ref_bin = cv2.threshold(ref_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

test_src = cv2.imread("./data/testShapes.jpg")
test_gray = cv2.cvtColor(test_src , cv2.COLOR_BGR2GRAY)
ret , test_bin = cv2.threshold(test_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)\

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

ref_contours, _ = cv2.findContours(ref_bin, mode , method)
test_contours, _ = cv2.findContours(test_bin, mode , method)

ref_dst = ref_src.copy()
colors = ((0,0,255),(0,255,0),(255,0,0))

for i , cnt in enumerate(ref_contours):
    cv2.drawContours(ref_dst, [cnt], 0, colors[i], 2)

#여기서 매칭을 한다.
test_dst = test_src.copy()
method = cv2.CONTOURS_MATCH_I1
for i , cnt in enumerate(test_contours):
    matches = []
    
    for cnt2 in ref_contours:
        ret = cv2.matchShapes(cnt , cnt2, method, 0)
        matches.append(ret)
    k = np.argmin(matches) # matches에는 다 비교한 결과가 들어가고 그 중에 가장 작은 값이 K 에 들어간다
    cv2.drawContours(test_dst, [cnt], 0, colors[k], 2)
    
cv2.imshow("ref_dst", ref_dst)
cv2.imshow("test_dst", test_dst)
cv2.waitKey()
cv2.destroyAllWindows()